In [16]:
import pandas as pd , os, numpy as np, matplotlib.pyplot as plt, glob, keras
from nltk.tokenize import regexp_tokenize ,word_tokenize ,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.moses import MosesDetokenizer
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Flatten, Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [2]:
fpath = glob.glob(os.path.join(os.getcwd(),'smsspam/*'))[0]      
with open(fpath) as f:
    print '***************************************************************'
    print f.readline()
    print '***************************************************************'
    lines = [line.strip().split('\t') for  line in f.readlines()]
y, X = [ int(line[0]=='spam')for line in lines] , [line[1] for line in lines]       


***************************************************************
ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

***************************************************************


In [3]:
wordvector_idx = {}
fpath = os.path.join(os.getcwd(),'glove','glove.6B.50d.txt')
with open(fpath) as f:
    for line in f.readlines():
        word = line.split()
        wordvector_idx[word[0]] = np.asarray( word[1:], dtype='float32')

In [5]:
stop = set(stopwords.words('english'))
detokenizer = MosesDetokenizer()
lem = WordNetLemmatizer()
X1= [detokenizer.detokenize([lem.lemmatize(i) for i in regexp_tokenize(X[list], r'\w+\.?|\d+') if i not in stop] ,return_str=True)  for list in range(len(X)) ] 

In [6]:
token = Tokenizer(num_words=11000)
token.fit_on_texts(X1)
sequences = token.texts_to_sequences(X1)
word2idx = token.word_index

In [7]:
len(word2idx) ;  len(wordvector_idx['get']); word2idx

{u'raining': 1543,
 u'yellow': 3853,
 u'narcotic': 4354,
 u'four': 2228,
 u'woods': 6463,
 u'hanging': 1903,
 u'looking': 394,
 u'electricity': 3599,
 u'scold': 3601,
 u'lord': 6547,
 u'rp176781': 4420,
 u'digit': 5465,
 u'callin': 2387,
 u'ew': 6472,
 u'hearin': 8000,
 u'screaming': 1558,
 u'disturb': 1060,
 u'prize': 105,
 u'andre': 8136,
 u'smsing': 7647,
 u'wednesday': 1232,
 u'oooh': 3082,
 u'specially': 1042,
 u'nigh': 7226,
 u'tired': 775,
 u'corvette': 4790,
 u'miller': 5396,
 u'clubmoby': 7796,
 u'second': 542,
 u'attended': 7430,
 u'txtno': 3012,
 u'scraped': 7836,
 u'2kbsubject': 4692,
 u'scallies': 7127,
 u'laxinorficated': 6859,
 u'cooking': 2152,
 u'fingers': 1772,
 u'maraikara': 6565,
 u'hero': 4941,
 u'y87': 6652,
 u'here': 486,
 u'specialise': 5482,
 u'47': 7417,
 u'china': 2688,
 u'dogwood': 7631,
 u'dorm': 3137,
 u'08718711108': 4627,
 u'previews': 5712,
 u'84122': 5047,
 u'w111wx': 2136,
 u'kids': 1488,
 u'84128': 2531,
 u'eastenders': 3291,
 u'09058091870': 7552,
 

In [8]:
def pretrained_emb():
    emb = np.zeros((len(word2idx) + 1,50))
    idx2word = {}
    idx2word = {idx : word for word, idx in word2idx.items()}
    idx_list =  []
    idx_list = [idx for word, idx in word2idx.items()]
    for idx in idx_list:
        word = idx2word[idx]
        if not np.any(wordvector_idx.get(word) == None):
            
            emb[idx] = wordvector_idx.get(word)
        else:
            emb[idx]=np.random.normal(scale=1, size=(50,))
        emb[-1]=np.random.normal(scale=1, size=(50,))    
    return emb

In [9]:
emb = pretrained_emb()

In [24]:
X_train, X_test,y_train, y_test = train_test_split(X2, y,random_state=57,shuffle=True,stratify=y,test_size=0.3 )

In [26]:
x_train = pad_sequences(X_train, maxlen=150)
x_test = pad_sequences(X_test, maxlen=150)

In [10]:
embedding_layer = Embedding(len(word2idx) +1 ,
                           50,
                           weights=[emb],
                           input_length=150,
                           trainable=False)

In [11]:
sequence_input = Input(shape=(150,), dtype='int32')

In [13]:
embedded_sequences = embedding_layer(sequence_input)

In [27]:
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
#x = Conv1D(128, 5, activation='relu')(x)
#x = MaxPooling1D(5)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(1, activation='sigmoid')(x)

In [28]:
#sgd = keras.optimizers.SGD(lr=0.001)

In [31]:
model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer= 'RMSprop',
              metrics=['acc'])

In [32]:
model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(X_test, y_test))

Train on 3901 samples, validate on 1672 samples
Epoch 1/10
3901/3901 [==============================] - 4s - loss: 0.2376 - acc: 0.9251 - val_loss: 0.1133 - val_acc: 0.9677